In [1]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, AveragePooling2D, Add, Activation, BatchNormalization, Concatenate, Dropout
from keras.layers import Input
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.models import Model
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import keras.backend as K

Using TensorFlow backend.
/home/haoran/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train = pd.read_json('data/train/processed/train.json')
test = pd.read_json('data/test/processed/test.json')
train['band_1'] = train['band_1'].apply(lambda x: np.reshape(x, [75,75]))
train['band_2'] = train['band_2'].apply(lambda x: np.reshape(x, [75,75]))
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors = 'coerce')
train['inc_angle'] = train['inc_angle'].fillna(value = train['inc_angle'].mean())

dataInput = []
for i in train.index:
    dataInput.append(np.stack([train.loc[i]['band_1'], train.loc[i]['band_2']], axis = -1))
dataInput = np.array(dataInput)

In [3]:
def createGenerator( X, I, Y):
    while True:
        idx = np.random.permutation( X.shape[0])
        datagen = image.ImageDataGenerator( 
                                            rotation_range=20,
                                            height_shift_range=0.1,                                           
                                            horizontal_flip = True,
                                           vertical_flip = True, 
                                           width_shift_range=0.1,
                                            fill_mode='wrap',
                                          )
      

        batches = datagen.flow( X[idx], Y[idx], batch_size=64, shuffle=False)
        idx0 = 0
        for batch in batches:
            idx1 = idx0 + batch[0].shape[0]

            yield [batch[0], I[ idx[ idx0:idx1 ] ]], batch[1]

            idx0 = idx1
            if idx1 >= X.shape[0]:
                break

In [4]:
def model1():
    image_input = Input(shape = (75,75,2))
    x = BatchNormalization(axis = -1, input_shape= [75,75,2])(image_input)
    #CNN 0
    x = Conv2D(filters = 32, kernel_size = (3,3), activation='relu')(x)
    x = BatchNormalization(axis = -1)(x) 
    x = MaxPooling2D((2,2), strides = (1,1))(x)

    #CNN 1
    x = Conv2D(filters = 64, kernel_size = (3,3), activation='relu')(x)
    x = BatchNormalization(axis = -1)(x) 
    x = MaxPooling2D((2,2))(x)

    #CNN 2
    x = Conv2D(filters = 128, kernel_size = (3,3), activation='relu')(x)
    x = BatchNormalization(axis = -1)(x) 
    x = MaxPooling2D((2,2))(x)

    #CNN 3
    x = Conv2D(filters = 128, kernel_size = (3,3), activation='relu')(x)
    x = BatchNormalization(axis = -1)(x) 
    x = MaxPooling2D((2,2))(x)

    #CNN 4
    x = Conv2D(filters = 64, kernel_size = (3,3), activation='relu')(x)
    x = BatchNormalization(axis = -1)(x) 
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.1)(x)

    x = Flatten()(x)
    inc_input = Input(shape = (1,)) #incidence angle
    y = BatchNormalization(axis = -1)(inc_input)
    x = Concatenate()([x,y])
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(1, activation='sigmoid')(x)    
    model = Model(inputs = [image_input, inc_input], outputs= x)
    return model

In [5]:
def identity_block(input_tensor, f, filters):
    f1,f2,f3 = filters
    x = Conv2D(f1, (1,1))(input_tensor)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(f2, f, padding = 'same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(f3, (1,1))(x)
    x = BatchNormalization(axis = -1)(x)
    
    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, f, filters, strides = (2,2)):
    f1,f2,f3 = filters
    x = Conv2D(f1, (1,1), strides = strides)(input_tensor)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(f2, f, padding = 'same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(f3, (1,1))(x)
    x = BatchNormalization(axis = -1)(x)
    
    shortcut = Conv2D(f3, (1,1), strides = strides)(input_tensor)
    shortcut = BatchNormalization(axis = -1)(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x


def model2():
    image_input = Input(shape = (75,75,2))
    x = BatchNormalization(axis = -1, input_shape= [75,75,2])(image_input)
    x = Conv2D(filters = 32, kernel_size = (3,3), activation='relu')(x)
    x = BatchNormalization(axis = -1)(x) 
    x = MaxPooling2D((2,2))(x)

    x = conv_block(x, 3, [64, 64, 256])
    x = identity_block(x, 3, [64, 64, 256])
    x = identity_block(x, 3, [64, 64, 256])

    x = conv_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])

    x = conv_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    #x = identity_block(x, 3, [256, 256, 1024])
    #x = identity_block(x, 3, [256, 256, 1024])
    #x = identity_block(x, 3, [256, 256, 1024])

    # x = conv_block(x, 3, [512, 512, 2048])
    # x = identity_block(x, 3, [512, 512, 2048])
    # x = identity_block(x, 3, [512, 512, 2048])

    x = AveragePooling2D((5,5))(x)

    x = Flatten()(x)
    inc_input = Input(shape = (1,)) #incidence angle
    y = BatchNormalization(axis = -1)(inc_input)
    x = Concatenate()([x,y])
    # x = Dense(512, activation='relu')(x)
    # x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs = [image_input, inc_input], outputs= x)
    return model

In [6]:
def trainModel(model, counter):
    model.compile(Adam(1e-5), loss = 'binary_crossentropy', metrics=['accuracy'])
    earlyStopping = EarlyStopping(monitor='loss', patience=50, verbose=0, mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=8, verbose=0, epsilon=1e-4, mode='min')
    model.fit_generator(createGenerator(X = dataInput, I = train.inc_angle.values, Y = train.is_iceberg.values),
                    steps_per_epoch = len(train)/64, epochs = 4, verbose = False)
    K.set_value(model.optimizer.lr, 1e-2)
    model.fit_generator(createGenerator(X = dataInput, I = train.inc_angle.values, Y = train.is_iceberg.values),
                         callbacks=[earlyStopping, reduce_lr_loss],
                       steps_per_epoch = len(train)/64, epochs = 300, verbose = False)
    print('Finished training model', counter)

In [8]:
cnnModels = [model1() for i in range(25)]
resModels = [model2() for i in range(10)]

In [9]:
for i in range(len(cnnModels)):
    trainModel(cnnModels[i], i)

/home/haoran/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1604, 75, 75, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Finished training model 0
Finished training model 1
Finished training model 2
Finished training model 3
Finished training model 4
Finished training model 5
Finished training model 6
Finished training model 7
Finished training model 8
Finished training model 9
Finished training model 10
Finished training model 11
Finished training model 12
Finished training model 13
Finished training model 14
Finished training model 15
Finished training model 16
Finished training model 17
Finished training model 18
Finished training model 19
Finished training model 20
Finished training model 21
Finished training model 22
Finished training model 23
Finished training model 24


In [10]:
for i in range(len(resModels)):
    trainModel(resModels[i], i)

/home/haoran/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1604, 75, 75, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Finished training model 0
Finished training model 1
Finished training model 2
Finished training model 3
Finished training model 4
Finished training model 5
Finished training model 6
Finished training model 7
Finished training model 8
Finished training model 9


In [11]:
for i in range(len(cnnModels)):
    cnnModels[i].save_weights(r'weights/ensembleFinal/cnn'+str(i)+'.hdf5')

In [12]:
for i in range(len(resModels)):
    resModels[i].save_weights(r'weights/ensembleFinal/res'+str(i)+'.hdf5')

In [17]:
test['band_1'] = test['band_1'].apply(lambda x: np.reshape(x, [75,75]))
test['band_2'] = test['band_2'].apply(lambda x: np.reshape(x, [75,75]))
dataTest = []
for i in test.index:
    dataTest.append(np.stack([test.loc[i]['band_1'], test.loc[i]['band_2']], axis = -1))
dataTest = np.array(dataTest)

In [18]:
predictions1 = [i.predict([dataTest, test.inc_angle.values]) for i in cnnModels]
predictions2 = [i.predict([dataTest, test.inc_angle.values]) for i in resModels]

In [19]:
pred_avg1 = np.mean(list(zip(*predictions1)),axis = 1)
pd.DataFrame({'id': test['id'], 'is_iceberg': np.ravel(pred_avg1)}).to_csv('data/answers/ensembleFinalCNNOnly.csv', index = False)

In [20]:
pred_avg2 = np.mean(list(zip(*predictions2)),axis = 1)
pd.DataFrame({'id': test['id'], 'is_iceberg': np.ravel(pred_avg2)}).to_csv('data/answers/ensembleFinalResOnly.csv', index = False)

In [26]:
pred_avg3 = np.mean([pred_avg1, pred_avg2], axis = 0)

In [29]:
pd.DataFrame({'id': test['id'], 'is_iceberg': np.ravel(pred_avg3)}).to_csv('data/answers/ensembleFinalCombined.csv', index = False)